# Baseball Stats Prediction Model

To predict future season stats for baseball players a player will generate next season, we'll first download baseball season data using pybaseball and clean it.
Link to the [data](https://www.fangraphs.com/players/shohei-ohtani/19755/stats?position=DH)


In [6]:
import os
import pandas as pd
import numpy as np
from pybaseball import batting_stats

In [7]:
START = 2002
END = 2022

In [8]:
if os.path.exists("batting.csv"):
    batting = pd.read_csv("batting.csv", index_col=0)
else:
    batting = batting_stats(START, END, qual=200)
    batting.to_csv("batting.csv")

Since we want to make predictions for players with data for more than one season, we will group the data based on playerid (IDfg) and remove any groups that have only one season of data.

In [9]:
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x:x.shape[0] > 1)

In [10]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN
3,15640,2022,Aaron Judge,NYY,30,144,533,641,169,82,...,0.264,118.4,227.0,0.594,382,0.174,0.290,NaN,NaN,NaN
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,0.131,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865,1698,2010,Gerald Laird,DET,30,89,270,299,56,40,...,NaN,NaN,0.0,NaN,0,0.166,0.252,NaN,NaN,NaN
7027,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,0.096,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN
6666,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN
6970,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN


Creating a Target Group


We will use the wins Above Replacement value (WAR) to predict the model. Definition [source](https://library.fangraphs.com/misc/war)

In [11]:
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

In [12]:
batting[["Name", "Season", "WAR", "Next_WAR"]]

,Name,Season,WAR,Next_WAR
5551,Alfredo Amezaga,2006,1.1,2.0
5001,Alfredo Amezaga,2007,2.0,1.2
5246,Alfredo Amezaga,2008,1.2,NaN
1165,Garret Anderson,2002,3.7,5.1
867,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6032,Owen Miller,2022,0.5,NaN
4880,Andrew Vaughn,2021,-0.3,0.4
2171,Andrew Vaughn,2022,0.4,NaN
6612,Ha-seong Kim,2021,0.5,3.0


We can see that there are missing values for our Next WAR and may be the player did not play or the he had no data to report for that year. 

Cleaning the data for Prediction.

In [13]:
null_count = batting.isnull().sum()
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
CSW%           0
xBA         6742
xSLG        6742
xwOBA       6742
Next_WAR    1175
Length: 320, dtype: int64

Removing all the columns with missing values

In [14]:
complete_cols = list(batting.columns[null_count == 0])

In [15]:
batting = batting[complete_cols + ["Next_WAR"]].copy()
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR
5551,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,86,107,113,143,109,63,0,0.188,0.256,2.0
5001,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,92,101,112,109,113,75,0,0.175,0.227,1.2
5246,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,99,101,101,123,111,64,0,0.178,0.244,NaN
1165,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,118,91,80,65,97,129,0,0.137,0.232,5.1
867,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,112,101,80,90,99,109,0,0.164,0.252,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,24655,2022,Owen Miller,CLE,25,123,406,451,100,69,...,93,109,99,130,102,81,327,0.189,0.268,NaN
4880,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,87,104,116,84,99,110,321,0.185,0.285,0.4
2171,26197,2022,Andrew Vaughn,CHW,24,124,476,518,136,90,...,88,108,108,95,98,106,395,0.203,0.287,NaN
6612,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,126,99,59,137,96,88,201,0.216,0.303,3.0


Convert data types to Numbers

In [16]:
batting.dtypes

IDfg          int64
Season        int64
Name         object
Team         object
Age           int64
             ...   
Hard%+        int64
Events        int64
CStr%       float64
CSW%        float64
Next_WAR    float64
Length: 132, dtype: object

In [17]:
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [18]:
del batting["Dol"]
del batting["Age Rng"]

In [19]:
batting["team_code"] = batting["Team"].astype("category").cat.codes


Copy to save the data for future and drop all NA Next_WAR rows

In [44]:
batting_full = batting.copy()
batting = batting.dropna()

Build the ML algorithm

In [21]:
from turtle import forward
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha = 1)

split = TimeSeriesSplit(n_splits = 3)

sfs = SequentialFeatureSelector(rr, n_features_to_select = 20, direction="forward", cv=split, n_jobs=6)

The alpha works like lambda and we initialise to 1. 
The Split will spit the data into 3 parts.
sfs is will evaluate the features forward till it gets to 20.

In [22]:
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

To scale the data so that the mean is 0 and the sd is 1, use the MinMaxSCaler. Select all rows(:), and the selected columns.

In [23]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

C:\Users\User\AppData\Local\Temp\ipykernel_25136\2781320675.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])


If you run into any error, run the .copy from the latest changes as we are manipulating the data too many times. .copy updates the original file.

In [24]:
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR,team_code
count,5567.000000,5567.00000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,...,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000,5567.000000
mean,5354.670918,2011.15035,0.360669,0.652953,0.478919,0.481222,0.366204,0.290644,0.399516,0.103490,...,0.457526,0.403242,0.410811,0.511029,0.478703,0.172653,0.498848,0.545712,1.790336,0.474213
std,5124.355355,5.60557,0.147487,0.255854,0.242344,0.262145,0.182502,0.138757,0.171704,0.105905,...,0.114056,0.131243,0.121116,0.130391,0.134055,0.273812,0.137231,0.120648,1.992286,0.305071
min,1.000000,2002.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.400000,0.000000
25%,1129.000000,2006.00000,0.269231,0.478632,0.276978,0.259516,0.211207,0.179245,0.258621,0.043478,...,0.382022,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.300000,0.205882
50%,3531.000000,2011.00000,0.346154,0.709402,0.507194,0.508651,0.370690,0.287736,0.379310,0.086957,...,0.460674,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,1.500000,0.470588
75%,8722.000000,2016.00000,0.461538,0.871795,0.688849,0.711073,0.508621,0.391509,0.517241,0.130435,...,0.528090,0.488722,0.483146,0.594203,0.564626,0.345576,0.591489,0.625551,2.900000,0.735294
max,27506.000000,2021.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


Apply the sfs to the data ie fit and pick the 20 predictors that give the greatest accuracy with the Ridge Regression model.

In [25]:
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=6)

To get the list of the columns that we want to use as predictors, use the index as  get_support()

In [26]:
predictors = list(selected_columns[sfs.get_support()])
predictors

['Age',
 'IBB',
 'SO',
 'SB',
 'BU',
 'BABIP',
 'IFH%',
 'WAR',
 'Spd',
 'PH',
 'CB%',
 'Z-Contact%',
 'SwStr%',
 'wGDP',
 'Oppo%',
 'OBP+',
 'SLG+',
 'Pull%+',
 'Soft%+',
 'Hard%+']

Create a new list of predictions for a single season. Years is a single season. Starting from 2007 (i), we using trained data from 2002-2006 to create predictions for 2007 etc till 2021. 

In [27]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []
    
    years = sorted(data["Season"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        #training set is anything before the current year
        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]
        
        model.fit(train[predictors], train["Next_WAR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        # compair the two values the actual vs prediction
        # when axis is 1, it creates two columns when concatenating
        combined = pd.concat([test["Next_WAR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined) 
    
    return pd.concat(all_predictions)

In [28]:
predictions = backtest(batting, rr, predictors)
predictions

,actual,prediction
5001,1.2,1.482739
1924,1.4,0.777661
3110,-0.1,0.561445
5787,0.6,0.902726
1103,4.8,2.262996
...,...,...
1914,1.5,2.731408
5865,1.0,1.933071
7016,0.5,1.533851
4880,0.4,1.700004


We can see that the algorithm is fairly good, we need to find how high the error is.

In [29]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

2.7780930725959636

In [30]:
batting["Next_WAR"].describe()

count    5567.000000
mean        1.790336
std         1.992286
min        -3.400000
25%         0.300000
50%         1.500000
75%         2.900000
max        11.900000
Name: Next_WAR, dtype: float64

Improving the model using player history

In [38]:
def player_history(df):
    df = df.sort_values("Season")
    
    df["player_season"] = range(0, df.shape[0])
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
    df["war_corr"].fillna(0, inplace=True)
    
    df["war_diff"] = df["WAR"] / df["WAR"].shift(1)
    df["war_diff"].fillna(1, inplace=True)
    # to prevent division by 0, we use np.inf
    df["war_diff"][df["war_diff"] == np.inf] = 1
    
    return df


Calling in the player history for each player

In [47]:
batting = batting.groupby("IDfg", group_keys=False).apply(player_history)

In [50]:
def group_averages(df):
    return df["WAR"] / df["WAR"].mean()

Group the averages by season

In [51]:
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

In [52]:
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]


In [53]:
predictions = backtest(batting, rr, new_predictors)

In [54]:
mean_squared_error(predictions["actual"], predictions["prediction"]) 

2.6856312201475796

We have improved the new predictors

In [55]:
pd.Series(rr.coef_, index=new_predictors).sort_values()

Age             -2.689771
WAR             -1.964402
BABIP           -1.621807
Soft%+          -1.221675
SLG+            -1.167358
SwStr%          -1.097362
BU              -1.002294
Z-Contact%      -0.729677
PH              -0.728277
SO              -0.675640
war_diff        -0.590645
wGDP            -0.489259
Pull%+          -0.256786
CB%             -0.256246
OBP+            -0.217113
war_corr        -0.089197
player_season    0.008325
IFH%             0.410585
Oppo%            0.624413
Spd              0.707879
SB               1.057886
IBB              1.861357
Hard%+           2.265552
war_season       3.499913
dtype: float64

To look at the difference between the actual vs prediction

In [59]:
diff = predictions["actual"] - predictions["prediction"]

In [60]:
merged = predictions.merge(batting, left_index=True, right_index=True)
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()


In [61]:
merged[["IDfg", "Season", "Name", "WAR", "Next_WAR", "diff"]].sort_values(["diff"])


,IDfg,Season,Name,WAR,Next_WAR,diff
3529,13066,2019,Teoscar Hernandez,0.291925,1.5,0.000046
4505,8347,2016,Denard Span,0.279503,0.7,0.000369
5753,9848,2016,Austin Jackson,0.236025,1.8,0.000505
2748,5887,2016,John Jaso,0.248447,-0.1,0.000615
3122,13359,2019,Tyler Naquin,0.285714,1.1,0.000824
...,...,...,...,...,...,...
3249,5631,2010,Matt Kemp,0.211180,8.3,6.372236
3826,1875,2009,Josh Hamilton,0.291925,8.4,6.598137
874,9166,2010,Buster Posey,0.459627,10.1,6.612272
450,15640,2021,Aaron Judge,0.552795,10.7,6.895011


More information needs to be included to make more accurate predictions like player injury, new players with little player information.